# Deriving Coupled Cluster Equations using Sympy

In [66]:
from sympy.physics.secondquant import (AntiSymmetricTensor, wicks,
        F, Fd, NO, evaluate_deltas, substitute_dummies, Commutator,
        simplify_index_permutations, PermutationOperator)
from sympy import (
    symbols, Rational, latex, Dummy
)
import re

In [15]:
pretty_dummies_dict = {
    'above': 'cdefgh',
    'below': 'klmno',
    'general': 'pqrstu'
}

## Define indexes

In [6]:
i = symbols('i', below_fermi=True, cls=Dummy)
a = symbols('a', above_fermi=True, cls=Dummy)
j = symbols('j', below_fermi=True, cls=Dummy)
b = symbols('b', above_fermi=True, cls=Dummy)
p, q, r, s = symbols('p,q,r,s', cls=Dummy)

## Define Hamiltonian

In [7]:
fock = AntiSymmetricTensor('f', (p,), (q,))
pr = NO(Fd(p)*F(q))
V = AntiSymmetricTensor('v',(p,q),(r,s))
pqsr = NO(Fd(p)*Fd(q)*F(s)*F(r))
H = fock*pr + Rational(1,4)*V*pqsr
H

AntiSymmetricTensor(f, (_p,), (_q,))*NO(CreateFermion(_p)*AnnihilateFermion(_q)) - AntiSymmetricTensor(v, (_p, _q), (_r, _s))*NO(CreateFermion(_p)*CreateFermion(_q)*AnnihilateFermion(_r)*AnnihilateFermion(_s))/4

## Define Cluster Operator

In [8]:
def get_T():
    i, j = symbols('i,j', below_fermi=True, cls=Dummy)
    a, b = symbols('a,b', above_fermi=True, cls=Dummy)
    t_ai = AntiSymmetricTensor('t', (a,), (i,))*NO(Fd(a)*F(i))
    t_abij = Rational(1,4)*AntiSymmetricTensor('t', (a,b), (i,j))*NO(Fd(a)*Fd(b)*F(j)*F(i))
    return t_ai + t_abij

## Perform the Hausdorff expansion

In [9]:
C = Commutator
T = get_T()
print("commutator 1...")
comm1 = wicks(C(H, T))
comm1 = evaluate_deltas(comm1)
comm1 = substitute_dummies(comm1)

commutator 1...


In [10]:
T = get_T()
print("commutator 2...")
comm2 = wicks(C(comm1, T))
comm2 = evaluate_deltas(comm2)
comm2 = substitute_dummies(comm2)

commutator 2...


In [11]:
T = get_T()
print("commutator 3...")
comm3 = wicks(C(comm2, T))
comm3 = evaluate_deltas(comm3)
comm3 = substitute_dummies(comm3)

commutator 3...


In [12]:
T = get_T()
print("commutator 4...")
comm4 = wicks(C(comm3, T))
comm4 = evaluate_deltas(comm4)
comm4 = substitute_dummies(comm4)

commutator 4...


## Construct the Similarty Transformed Hamiltonian

In [17]:
eq = H + comm1 + comm2/2 + comm3/6 + comm4/24
eq = eq.expand()
eq = evaluate_deltas(eq)
eq = substitute_dummies(eq, new_indices=True,
        pretty_indices=pretty_dummies_dict)

## Get energy expression

In [21]:
i, j, k, l = symbols('i,j,k,l', below_fermi=True)
a, b, c, d = symbols('a,b,c,d', above_fermi=True)
print()
print("CC Energy:")
energy = wicks(eq, simplify_dummies=True,
        keep_only_fully_contracted=True)
latex(energy)


CC Energy:


'f^{k}_{c} t^{c}_{k} - \\frac{t^{c}_{l} t^{d}_{k} v^{kl}_{cd}}{2} + \\frac{t^{cd}_{kl} v^{kl}_{cd}}{4}'

## Get amplitude Equations

In [22]:
eqT1 = wicks(NO(Fd(i)*F(a))*eq, simplify_kronecker_deltas=True, keep_only_fully_contracted=True)
eqT1 = substitute_dummies(eqT1)
latex(eqT1)

'- f^{k}_{c} t^{c}_{i} t^{a}_{k} + f^{k}_{c} t^{ac}_{ik} - f^{k}_{i} t^{a}_{k} + f^{a}_{c} t^{c}_{i} + f^{a}_{i} - t^{c}_{k} t^{d}_{i} t^{a}_{l} v^{kl}_{cd} - t^{c}_{k} t^{d}_{i} v^{ak}_{cd} + t^{c}_{k} t^{a}_{l} v^{kl}_{ic} + t^{c}_{k} t^{ad}_{il} v^{kl}_{cd} + t^{c}_{k} v^{ak}_{ic} - \\frac{t^{c}_{i} t^{ad}_{kl} v^{kl}_{cd}}{2} + \\frac{t^{a}_{l} t^{cd}_{ik} v^{kl}_{cd}}{2} + \\frac{t^{cd}_{ik} v^{ak}_{cd}}{2} - \\frac{t^{ac}_{kl} v^{kl}_{ic}}{2}'

In [25]:
eqT2 = wicks(NO(Fd(i)*Fd(j)*F(b)*F(a))*eq, simplify_dummies=True, 
             keep_only_fully_contracted=True, simplify_kronecker_deltas=True)
x = latex(eqT2)

In [26]:
type(x)

str

In [27]:
x

'f^{k}_{c} t^{c}_{i} t^{ab}_{jk} - f^{k}_{c} t^{c}_{j} t^{ab}_{ik} + f^{k}_{c} t^{a}_{k} t^{bc}_{ij} - f^{k}_{c} t^{b}_{k} t^{ac}_{ij} + f^{k}_{i} t^{ab}_{jk} - f^{k}_{j} t^{ab}_{ik} - f^{a}_{c} t^{bc}_{ij} + f^{b}_{c} t^{ac}_{ij} + t^{c}_{k} t^{d}_{i} t^{ab}_{jl} v^{kl}_{cd} - t^{c}_{k} t^{d}_{j} t^{ab}_{il} v^{kl}_{cd} + t^{c}_{k} t^{a}_{l} t^{bd}_{ij} v^{kl}_{cd} - t^{c}_{k} t^{b}_{l} t^{ad}_{ij} v^{kl}_{cd} - t^{c}_{k} t^{ad}_{ij} v^{bk}_{cd} + t^{c}_{k} t^{ab}_{il} v^{kl}_{jc} - t^{c}_{k} t^{ab}_{jl} v^{kl}_{ic} + t^{c}_{k} t^{bd}_{ij} v^{ak}_{cd} + t^{c}_{i} t^{d}_{j} t^{a}_{k} t^{b}_{l} v^{kl}_{cd} + t^{c}_{i} t^{d}_{j} t^{a}_{k} v^{bk}_{cd} - t^{c}_{i} t^{d}_{j} t^{b}_{k} v^{ak}_{cd} + \\frac{t^{c}_{i} t^{d}_{j} t^{ab}_{kl} v^{kl}_{cd}}{2} + t^{c}_{i} t^{d}_{j} v^{ab}_{cd} - t^{c}_{i} t^{a}_{k} t^{b}_{l} v^{kl}_{jc} - t^{c}_{i} t^{a}_{k} t^{bd}_{jl} v^{kl}_{cd} - t^{c}_{i} t^{a}_{k} v^{bk}_{jc} + t^{c}_{i} t^{b}_{k} v^{ak}_{jc} - t^{c}_{i} t^{b}_{l} t^{ad}_{jk} v^{kl}_{cd} - t^

In [30]:
xs = x.split()

In [31]:
xs[0].replace('{\w}','()')

'f^{k}_{c}'

In [32]:
# Given a latex string, make it into my style of string

In [99]:
def convert(inp):
    # Remove spaces betweem signs and next terms
    out = re.sub('([+-])\s', '\\1', inp)
    # Add * between tensors contraction
    out = re.sub('}\s+?(\w)', '}*\\1', out)
    # Convert the index to my parentesis notation
    out = re.sub('{(\w+?)}_{(\w+?)}', '(\\1,\\2)', out)
    # Convert tensor names
    out = re.sub('f\^', 'f', out)
    out = re.sub('t\^', 'T', out)
    out = re.sub('v\^', 'A', out)
    # Remove fractions
    out = re.sub('\\\\frac{(.+?)}{(\d+?)}', '(1/\\2)*\\1', out)
    return out

In [100]:
convert(x)

'f(k,c)*T(c,i)*T(ab,jk) -f(k,c)*T(c,j)*T(ab,ik) +f(k,c)*T(a,k)*T(bc,ij) -f(k,c)*T(b,k)*T(ac,ij) +f(k,i)*T(ab,jk) -f(k,j)*T(ab,ik) -f(a,c)*T(bc,ij) +f(b,c)*T(ac,ij) +T(c,k)*T(d,i)*T(ab,jl)*A(kl,cd) -T(c,k)*T(d,j)*T(ab,il)*A(kl,cd) +T(c,k)*T(a,l)*T(bd,ij)*A(kl,cd) -T(c,k)*T(b,l)*T(ad,ij)*A(kl,cd) -T(c,k)*T(ad,ij)*A(bk,cd) +T(c,k)*T(ab,il)*A(kl,jc) -T(c,k)*T(ab,jl)*A(kl,ic) +T(c,k)*T(bd,ij)*A(ak,cd) +T(c,i)*T(d,j)*T(a,k)*T(b,l)*A(kl,cd) +T(c,i)*T(d,j)*T(a,k)*A(bk,cd) -T(c,i)*T(d,j)*T(b,k)*A(ak,cd) +(1/2)*T(c,i)*T(d,j)*T(ab,kl)*A(kl,cd) +T(c,i)*T(d,j)*A(ab,cd) -T(c,i)*T(a,k)*T(b,l)*A(kl,jc) -T(c,i)*T(a,k)*T(bd,jl)*A(kl,cd) -T(c,i)*T(a,k)*A(bk,jc) +T(c,i)*T(b,k)*A(ak,jc) -T(c,i)*T(b,l)*T(ad,jk)*A(kl,cd) -T(c,i)*T(ad,jk)*A(bk,cd) -(1/2)*T(c,i)*T(ab,kl)*A(kl,jc) +T(c,i)*T(bd,jk)*A(ak,cd) -T(c,i)*A(ab,jc) +T(c,j)*T(a,k)*T(b,l)*A(kl,ic) +T(c,j)*T(a,k)*T(bd,il)*A(kl,cd) +T(c,j)*T(a,k)*A(bk,ic) -T(c,j)*T(b,k)*A(ak,ic) +T(c,j)*T(b,l)*T(ad,ik)*A(kl,cd) +T(c,j)*T(ad,ik)*A(bk,cd) +(1/2)*T(c,j)*T(ab,k

In [89]:
test = '\\frac{T(ac,ij)*T(bd,kl)*A(kl,cd)}{2}'

In [98]:
re.sub('\\\\frac{(.+?)}{(\d+?)}', '(1/\\2)*\\1', test)

'(1/2)*T(ac,ij)*T(bd,kl)*A(kl,cd)'